# Imports

In [1]:
# --- Google Cloud / Colab integration ---
from google.colab import userdata        # Colab secure storage (for project IDs etc.)
from google.cloud.bigquery import magics # BigQuery Jupyter/Colab magic commands

# --- Data handling ---
import numpy as np
import pandas as pd
import datetime as dt

# --- Machine Learning ---
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_recall_fscore_support,
    roc_auc_score
)

In [2]:
%load_ext bigquery_magics

In [5]:
# getting PROJECT ID from user secrets
PROJECT_ID = userdata.get('PROJECT_ID')

In [6]:
# setting PROJECT ID
magics.context.project = PROJECT_ID

# Functions

In [7]:
def evaluate(y_true, y_pred, y_prob=None, name="model"):
    """
    Evaluate classification performance on imbalanced data.

    Parameters
    ----------
    y_true : array-like
        Ground-truth labels (0/1).
    y_pred : array-like
        Binary predictions.
    y_prob : array-like, optional
        Predicted probabilities for the positive class (needed for AUC).
    name : str
        Model name (for printing).

    Returns
    -------
    dict
        Dictionary with accuracy, precision, recall, F1, and AUC.
    """
    # Accuracy (can be misleading under imbalance)
    acc = accuracy_score(y_true, y_pred)

    # Precision, recall, F1 (binary average, avoid div/0 errors)
    p, r, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="binary", zero_division=0
    )

    # AUC only if probabilities are available and more than one class in y_true
    auc = roc_auc_score(y_true, y_prob) if (y_prob is not None and len(np.unique(y_true)) > 1) else np.nan

    # Nicely formatted printout
    print(f"{name:>14} | acc={acc:.3f}  prec={p:.3f}  rec={r:.3f}  f1={f1:.3f}  auc={auc:.3f}")

    return {"acc": acc, "prec": p, "rec": r, "f1": f1, "auc": auc}

In [8]:
def best_threshold(y_true, y_prob):
    """
    Find the decision threshold that maximizes F1-score.

    Parameters
    ----------
    y_true : array-like
        Ground-truth binary labels (0/1).
    y_prob : array-like
        Predicted probabilities for the positive class.

    Returns
    -------
    t_star : float
        Threshold that yields the highest F1-score.
    f1_star : float
        Best F1-score obtained at that threshold.
    """
    ths = np.linspace(0.05, 0.95, 19)  # candidate thresholds
    scores = []

    for t in ths:
        pred = (y_prob >= t).astype(int)
        f1 = f1_score(y_true, pred, zero_division=0)
        scores.append((t, f1))

    # pick threshold with max F1
    t_star, f1_star = max(scores, key=lambda x: x[1])
    return t_star, f1_star

In [9]:
def station_from_dummies(row):
    """
    Recover the original station number from one-hot encoded station columns.

    Parameters
    ----------
    row : pandas.Series
        A row of the dataframe containing station dummy columns (e.g., 'station_725300').

    Returns
    -------
    str
        The station number as a string (e.g., '725300').
        Returns None if no station columns are present.
    """
    if not station_cols:  # safety: if no station dummies exist
        return None
    # index of the max dummy (should be exactly 1)
    ix = np.argmax(row[station_cols].values)
    # extract station id (remove "station_" prefix)
    return station_cols[ix].replace("station_", "")

# "Will it snow tomorrow?" - The time traveler asked
The following dataset contains climate information from over 9000 stations accross the world. The overall goal of these subtasks will be to predict whether it will snow tomorrow 20 years ago. So if today is 1 April 2025 then the weather we want to forecast is for the 2 April 2005. You are supposed to solve the tasks using Big Query, which can be used in the Jupyter Notebook like it is shown in the following cell. For further information and how to use BigQuery in Jupyter Notebook refer to the Google Docs.

The goal of this test is to test your coding knowledge in Python, BigQuery and Pandas as well as your understanding of Data Science. If you get stuck in the first part, you can use the replacement data provided in the second part.

In [10]:
%%bigquery
SELECT
*,
FROM `bigquery-public-data.samples.gsod`
LIMIT 20

Query is running:   0%|          |

Downloading:   0%|          |

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,33110,99999,1929,12,18,47.500000,4,44.000000,4,NaN,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
1,39730,99999,1929,11,19,47.799999,4,43.799999,4,NaN,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
2,30050,99999,1929,11,1,49.200001,4,45.500000,4,1019.900024,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
3,37950,99999,1929,10,20,43.500000,4,40.000000,4,1001.799988,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
4,30910,99999,1929,10,5,46.000000,4,NaN,<NA>,997.700012,...,NaN,<NA>,0.16,NaN,False,False,False,False,False,False
5,37770,99999,1929,8,19,57.799999,4,50.000000,4,1024.599976,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
6,39800,99999,1929,10,19,49.000000,4,46.900002,4,1007.500000,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
7,33790,99999,1929,12,1,41.799999,4,40.700001,4,994.200012,...,NaN,<NA>,0.00,NaN,True,True,True,True,True,True
8,33110,99999,1929,11,16,43.200001,4,40.500000,4,993.599976,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
9,34970,99999,1929,12,30,43.599998,5,40.700001,4,1002.900024,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False


## Part 1

### 1. Task
Change the date format to 'YYYY-MM-DD' and select the data from 2000 till 2005 for station numbers including and between 725300 and 726300 , and save it as a pandas dataframe. Note the maximum year available is 2010.

In [11]:
%%bigquery df
SELECT
  FORMAT_DATE('%Y-%m-%d', DATE(year, month, day)) AS date,
  *
FROM `bigquery-public-data.samples.gsod`
WHERE
  station_number BETWEEN 725300 AND 726300
  AND DATE(year, month, day) BETWEEN DATE('2000-01-01') AND DATE('2005-12-31')
ORDER BY station_number, date

Query is running:   0%|          |

Downloading:   0%|          |

In [12]:
df.shape

(447037, 32)

In [13]:
df.head()

,date,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,2000-01-01,725300,94846,2000,1,1,38.400002,24,29.700001,24,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
1,2000-01-02,725300,94846,2000,1,2,47.799999,24,42.299999,24,...,NaN,<NA>,0.01,NaN,True,True,True,True,True,True
2,2000-01-03,725300,94846,2000,1,3,37.500000,24,34.099998,24,...,NaN,<NA>,0.01,NaN,True,True,True,True,True,True
3,2000-01-04,725300,94846,2000,1,4,29.700001,24,25.900000,24,...,NaN,<NA>,0.25,1.2,True,True,True,True,True,True
4,2000-01-05,725300,94846,2000,1,5,20.400000,24,16.500000,24,...,NaN,<NA>,0.00,NaN,True,True,True,True,True,True


In [14]:
df.date.min(), df.date.max()

('2000-01-01', '2005-12-31')

In [15]:
df.station_number.min(), df.station_number.max()

(np.int64(725300), np.int64(726300))

### 2. Task
From here you want to work with the data from all stations 725300 to 725330 that have information from 2000 till 2005.

In [16]:
%%bigquery df
SELECT
  FORMAT_DATE('%Y-%m-%d', DATE(year, month, day)) AS date,
  *
FROM `bigquery-public-data.samples.gsod`
WHERE
  station_number BETWEEN 725300 AND 725330
  AND DATE(year, month, day) BETWEEN DATE('2000-01-01') AND DATE('2005-12-31')
ORDER BY station_number, date

Query is running:   0%|          |

Downloading:   0%|          |

In [17]:
df.shape

(21853, 32)

In [18]:
df.head()

,date,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,2000-01-01,725300,94846,2000,1,1,38.400002,24,29.700001,24,...,NaN,<NA>,0.00,NaN,False,False,False,False,False,False
1,2000-01-02,725300,94846,2000,1,2,47.799999,24,42.299999,24,...,NaN,<NA>,0.01,NaN,True,True,True,True,True,True
2,2000-01-03,725300,94846,2000,1,3,37.500000,24,34.099998,24,...,NaN,<NA>,0.01,NaN,True,True,True,True,True,True
3,2000-01-04,725300,94846,2000,1,4,29.700001,24,25.900000,24,...,NaN,<NA>,0.25,1.2,True,True,True,True,True,True
4,2000-01-05,725300,94846,2000,1,5,20.400000,24,16.500000,24,...,NaN,<NA>,0.00,NaN,True,True,True,True,True,True


In [19]:
# checking date
df.date.min(), df.date.max()

('2000-01-01', '2005-12-31')

In [20]:
# checking station number
df.station_number.min(), df.station_number.max()

(np.int64(725300), np.int64(725330))

Start by checking which year received the most snowfall in our data.

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21853 entries, 0 to 21852
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   date                                21853 non-null  object 
 1   station_number                      21853 non-null  Int64  
 2   wban_number                         21853 non-null  Int64  
 3   year                                21853 non-null  Int64  
 4   month                               21853 non-null  Int64  
 5   day                                 21853 non-null  Int64  
 6   mean_temp                           21853 non-null  float64
 7   num_mean_temp_samples               21853 non-null  Int64  
 8   mean_dew_point                      21846 non-null  float64
 9   num_mean_dew_point_samples          21846 non-null  Int64  
 10  mean_sealevel_pressure              19266 non-null  float64
 11  num_mean_sealevel_pressure_samples  19266

In [23]:
%%bigquery snow_by_year
SELECT
  EXTRACT(YEAR FROM DATE(year, month, day)) AS year, -- extract year from date
  SUM(CAST(snow_depth AS INT64))            AS snow_depth -- aggregate yearly snow depth
FROM `bigquery-public-data.samples.gsod`
WHERE
  station_number BETWEEN 725300 AND 725330
  AND DATE(year, month, day) BETWEEN DATE('2000-01-01') AND DATE('2005-12-31')
GROUP BY year
ORDER BY snow_depth DESC, year ASC

Query is running:   0%|          |

Downloading:   0%|          |

In [24]:
snow_by_year

,year,snow_depth
0,2000,1079
1,2001,584
2,2005,459
3,2004,378
4,2003,289
5,2002,276


Add an additional field that indicates the daily change in snow depth measured at every station. And identify the station and day for which the snow depth increased the most.  

In the dataset, the `snow_depth` column contains many missing values. To retain as much temporal information as possible, I decided to replace missing values with `0`. This way, we avoid dropping large parts of the time series and can still work with a complete dataset for modeling.

A potential refinement would be to distinguish between *true zero snow depth* and *missing measurements*. For example, if the boolean feature `snow` indicates snowfall on a given day but `snow_depth` is missing, we could add an additional indicator column (`snow_depth_reported`) to flag whether the value was actually measured or imputed. This would preserve information about measurement quality and could improve model performance.


In [27]:
%%bigquery df
WITH base AS ( -- base as a view
  SELECT
    FORMAT_DATE('%Y-%m-%d', DATE(year, month, day)) AS date,
    *,
    IFNULL(snow_depth, 0) AS snow_depth_clean -- Replace missing snow_depth with 0
  FROM `bigquery-public-data.samples.gsod`
  WHERE
    station_number BETWEEN 725300 AND 725330
    AND DATE(year, month, day) BETWEEN DATE('2000-01-01') AND DATE('2005-12-31')
),
diffs AS (
  SELECT
    *,
    snow_depth_clean - LAG(snow_depth_clean) OVER (PARTITION BY station_number ORDER BY date) AS daily_change_snow_depth -- Compute daily snow-depth change (lag window function)
  FROM base
)
SELECT
  *
FROM diffs
ORDER BY daily_change_snow_depth DESC

Query is running:   0%|          |

Downloading:   0%|          |

In [28]:
df.shape

(21853, 34)

In [29]:
df.head(1)

,date,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,...,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado,snow_depth_clean,daily_change_snow_depth
0,2005-01-22,725300,94846,2005,1,22,18.700001,24,14.6,24,...,0.5,11.8,True,True,True,True,True,True,11.8,9.8


The highest increase of snow depth was measured by station 725300 on Jan. 22nd  2005.

Do further checks on the remaining dataset, clean or drop data depending on how you see appropriate.

In [30]:
# setting date as datetime
df['date'] = pd.to_datetime(df['date'])

In [31]:
# sorting
df = df.sort_values(by=['station_number', 'date'])

In [32]:
# target: snow tomorrow (0/1)
df['snow_tomorrow'] = (
    df.groupby('station_number')['snow']
      .shift(-1)                     # tomorrow
      .astype('Int64')               # int
)

In [33]:
# Drop rows where target is NaN
df = df.dropna(subset=['snow_tomorrow'])

Since the dataset only contains 10 unique stations, I included station_number as a categorical feature via one-hot encoding. This allows the model to capture station-specific snowfall likelihoods. With a small number of categories this does not introduce a high-dimensional feature space.

In [34]:
# checking if all stations have enough data
df.groupby("station_number")["date"].agg(["min","max","count"])

,min,max,count
station_number,,,
725300,2000-01-01,2005-12-30,2191
725305,2000-01-01,2005-12-30,2188
725314,2000-01-01,2005-12-30,2175
725315,2000-01-01,2005-12-30,2191
725316,2000-01-01,2005-12-30,2186
725317,2000-01-01,2005-12-30,2181
725320,2000-01-01,2005-12-30,2191
725326,2000-01-01,2005-12-30,2185
725327,2000-01-01,2005-12-30,2164


In [35]:
# Station dummies
station_dummies = pd.get_dummies(df['station_number'], prefix='station')

# concat with df
df = pd.concat([df, station_dummies], axis=1)

# dropping original station_number
df = df.drop(columns=['station_number'])

To capture seasonality without introducing an artificial break between December (12) and January (1), we encode the month (1–12) as two cyclical features using sine and cosine transforms.

In [36]:
# Making a cyclical feature out of month (1–12)
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

We remove columns that are either:
- purely identifiers (`wban_number`, `year`, `month`, `day`),  
- almost completely missing (`mean_station_pressure`, `min_temperature`, `max_gust_wind_speed`),  
- redundant after cleaning (`snow_depth`, replaced by `snow_depth_clean`), or  
- quality counters not used here (`num_mean_*_samples`).  
This reduces noise and simplifies the feature set.

In [37]:
# dropping unnecessary columns or columns with too many NaN values
df = df.drop(columns=[
    "wban_number",
    "year",
    "month",
    "day",
    "mean_station_pressure",
    "num_mean_station_pressure_samples",
    "min_temperature",
    "min_temperature_explicit",
    "snow_depth",
    "max_gust_wind_speed",
    "num_mean_sealevel_pressure_samples"
])

I used median imputation as a pragmatic choice for small gaps. For precipitation, missing values were set to 0. A more refined approach would be time-based interpolation to better preserve local temporal structure.

In [38]:
# Simple imputation
for col in ['mean_dew_point', 'mean_wind_speed', 'max_temperature', 'total_precipitation', 'mean_sealevel_pressure', 'mean_visibility']:
    df[col] = df[col].fillna(df[col].median())

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21843 entries, 21851 to 2229
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         21843 non-null  datetime64[ns]
 1   mean_temp                    21843 non-null  float64       
 2   num_mean_temp_samples        21843 non-null  Int64         
 3   mean_dew_point               21843 non-null  float64       
 4   num_mean_dew_point_samples   21836 non-null  Int64         
 5   mean_sealevel_pressure       21843 non-null  float64       
 6   mean_visibility              21843 non-null  float64       
 7   num_mean_visibility_samples  21827 non-null  Int64         
 8   mean_wind_speed              21843 non-null  float64       
 9   num_mean_wind_speed_samples  21836 non-null  Int64         
 10  max_sustained_wind_speed     21835 non-null  float64       
 11  max_temperature              21843 non-null

In [40]:
# dropping the remained NaNs
df = df.dropna()

In [41]:
df.shape

(21801, 35)

### 3. Task
Now it is time to split the data, into a training, evaluation and test set. As a reminder, the date we are trying to predict snow fall for should constitute your test set.

In [42]:
str(dt.datetime.today()- dt.timedelta(days=20*365)).split(' ')[0]

'2005-08-29'

The official challenge asks us to predict tomorrow’s snowfall 20 years ago. To reflect this, I defined the test set as all data from the cutoff date 20 years ago (≈2005). Training and validation use only earlier years (2000-2004), ensuring no data leakage.

In [43]:
# Cutoff: today - 20 years
cutoff_date = (dt.datetime.today() - dt.timedelta(days=20*365)).date()
print("Target test date:", cutoff_date)

# Test: exact day
test_df = df[df["date"] == pd.to_datetime(cutoff_date)].copy()

# Train+Val: all data before
train_val_df = df[df["date"] < pd.to_datetime(cutoff_date)].copy()

Target test date: 2005-08-29


In [45]:
# train-val-split
train_df = train_val_df[train_val_df["date"] <= "2003-12-31"].copy()
val_df = train_val_df[(train_val_df["date"] > "2003-12-31") &
                      (train_val_df["date"] <= "2004-12-31")].copy()

In [46]:
print("Train:", train_df["date"].min(), "->", train_df["date"].max(), len(train_df))
print("Val:  ", val_df["date"].min(), "->", val_df["date"].max(), len(val_df))
print("Test:", test_df["date"].min(), "->", test_df["date"].max(), len(test_df))

Train: 2000-01-02 00:00:00 -> 2003-12-31 00:00:00 14535
Val:   2004-01-01 00:00:00 -> 2004-12-31 00:00:00 3650
Test: 2005-08-29 00:00:00 -> 2005-08-29 00:00:00 10


## Part 2
If you made it up to here all by yourself, you can use your prepared dataset to train an algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:

In [47]:
str(dt.datetime.today()- dt.timedelta(days=20*365)).split(' ')[0]

'2005-08-29'

You are allowed to use any library you are comfortable with such as sklearn, tensorflow, keras etc.
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1.

In [48]:
target = "snow_tomorrow"

In [49]:
# choose feature columns (everything numeric except date & target)
drop_cols = ["date", target]
X_cols = [c for c in train_df.columns if c not in drop_cols]

In [51]:
X_train, y_train = train_df[X_cols], train_df[target].astype(int)
X_val, y_val = val_df[X_cols], val_df[target].astype(int)
X_test = test_df[X_cols]
y_test = test_df[target].astype(int)

In [52]:
print("n_train:", len(X_train), "n_val:", len(X_val), "n_test:", len(X_test))
print("Positive rate (train/val):", y_train.mean().round(3), y_val.mean().round(3))

n_train: 14535 n_val: 3650 n_test: 10
Positive rate (train/val): 0.196 0.073


### Baseline

In [53]:
# Always-0 baseline
evaluate(y_val, np.zeros_like(y_val), name="always_0")

      always_0 | acc=0.927  prec=0.000  rec=0.000  f1=0.000  auc=nan


{'acc': 0.9265753424657535, 'prec': 0.0, 'rec': 0.0, 'f1': 0.0, 'auc': nan}

The naive baseline of always predicting ‘no snow tomorrow’ achieves ~93% accuracy due to class imbalance, but completely fails to identify snow days (F1 = 0, Recall = 0). This underlines the need for more informative models and alternative metrics such as recall, precision and F1.

In [54]:
# "snow today = snow tomorrow" baseline (uses today's snow as predictor)
evaluate(y_val, val_df["snow"].values.astype(int), name="snow_today")

    snow_today | acc=0.883  prec=0.195  rec=0.190  f1=0.193  auc=nan


{'acc': 0.883013698630137,
 'prec': 0.19540229885057472,
 'rec': 0.19029850746268656,
 'f1': 0.19281663516068054,
 'auc': nan}

The baseline of simply assuming that snowfall tomorrow equals snowfall today achieves ~88% accuracy. While worse than the trivial always-0 baseline in terms of accuracy, it at least captures ~19% of true snow days (recall). However, the precision and F1 remain low, highlighting that more sophisticated models are needed.

### Logistic Regression

I chose logistic regression as the first modeling approach because:
- It provides **probabilistic predictions** (needed for threshold tuning under class imbalance).
- It is **fast, simple, and interpretable**, making it an excellent benchmark against naive baselines.
- With `class_weight="balanced"`, it handles the skewed distribution of snow events reasonably well.
- It serves as a **baseline ML model** before moving on to more complex classifiers like Random Forest or algorithms of time series analysis.

In [55]:
# scaling
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

In [56]:
# Logistic Regression (with scaling, balanced classes)
logit = LogisticRegression(class_weight="balanced")
logit.fit(X_train_s, y_train)

LogisticRegression(class_weight='balanced')

In [57]:
val_prob_logit = logit.predict_proba(X_val_s)[:,1]

In [58]:
t_logit, _ = best_threshold(y_val, val_prob_logit)

#### Evaluation

In [59]:
# Validation performance
evaluate(y_val, (val_prob_logit>=t_logit).astype(int), val_prob_logit, name="logit")

         logit | acc=0.509  prec=0.101  rec=0.720  f1=0.177  auc=0.618


{'acc': 0.5093150684931507,
 'prec': 0.10110005238344683,
 'rec': 0.7201492537313433,
 'f1': 0.1773082223242995,
 'auc': np.float64(0.6175792386382694)}

- **Recall ~72%**: the model detects most snow days.  
- **Precision ~10%**: many false alarms, but this is expected under threshold tuning for high recall.  
- **F1 ~0.18, AUC ~0.62**: performance is modest but clearly above baselines (which failed to identify snow days at all).  

This demonstrates the classic **precision–recall trade-off**: the model is sensitive to rare snow events but at the cost of specificity.  
In practice, this is often acceptable if *missing snow is more costly than false alarms*.

In [65]:
test_prob_logit = logit.predict_proba(X_test_s)[:,1]

In [66]:
test_pred_logit = (test_prob_logit >= t_logit).astype(int)

In [67]:
# recover station id from dummies (if original station_number was dropped)
station_cols = [c for c in X_test.columns if c.startswith("station_")]

In [68]:
out = test_df.copy()
out["station"] = out.apply(station_from_dummies, axis=1)
out = out[["date", "station", "snow_tomorrow"]].copy()
out["p_logit"] = test_prob_logit
out["yhat_logit"] = test_pred_logit

In [69]:
out = out.sort_values("station").reset_index(drop=True)
out

,date,station,snow_tomorrow,p_logit,yhat_logit
0,2005-08-29,725300,0,0.676522,1
1,2005-08-29,725305,0,0.172101,0
2,2005-08-29,725314,0,0.168172,0
3,2005-08-29,725315,0,0.356000,1
4,2005-08-29,725316,0,0.086002,0
5,2005-08-29,725317,0,0.268302,1
6,2005-08-29,725320,0,0.607337,1
7,2005-08-29,725326,0,0.000592,0
8,2005-08-29,725327,0,0.141565,0
9,2005-08-29,725330,1,0.859215,1


- Out of 10 stations, only **station 725330** had snowfall on the following day.
- The logistic regression model **correctly identified this event** (recall = 100%).
- However, it also predicted snow for 4 stations where none occurred (false positives).

**Test-day metrics:**
- Accuracy = 60%  
- Precision = 20%  
- Recall = 100%  
- F1 = 0.33  

This outcome highlights the chosen strategy: maximize recall to avoid missing true snow events, accepting lower precision as a trade-off.